# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')

display(data_0)
data_0.info()
display(data_1)
data_1.info()
display(data_2)
data_2.info()
print(data_0['id'].value_counts().head(10))
print(data_1['id'].value_counts().head(4))
print(data_2['id'].value_counts().head(4))

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
TtcGQ    2
bsk9y    2
fiKDv    2
AGS9W    2
HZww2    2
74z30    2
QcMuo    2
bxg6G    2
Tdehs    2
A5aEY    2
Name: id, dtype: int64
LHZR0    2
bfPNe    2
5ltQ6    2
wt4Uk    2
Name: id, dtype: int64
xCHr8    2
Vcm5J    2
VF7Jo    2
KUPhW    2
Name: id, dtype: int64


In [3]:
display("Матрица корреляции для первой таблицы: ",
        data_0.corr(),
        "Матрица корреляции для второй таблицы: ",
        data_1.corr(),
        "Матрица корреляции для третьей таблицы: ",
        data_2.corr())

'Матрица корреляции для первой таблицы: '

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


'Матрица корреляции для второй таблицы: '

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


'Матрица корреляции для третьей таблицы: '

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


# Вывод:

С данными всё в порядке, пропусков не обнаружено типы данных соответствуют

Обнаружил совпадение id у 10, 4 и 4 месторождений для каждого из регионов, но так как сами признаки и целевой признак разные, то не буду удалять дубликаты.

Матрица корреляции показала, что признаки друг с другом не коррелируют, но вот между признаком f2 и целевым признаком видна сильная зависимость, во второй таблице и вовсе почти 1.

## Обучение и проверка модели

2.1. Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.

2.2. Обучите модель и сделайте предсказания на валидационной выборке.

2.3. Сохраните предсказания и правильные ответы на валидационной выборке.

2.4. Напечатайте на экране средний запас предсказанного сырья и RMSE модели.

2.5. Проанализируйте результаты.

In [4]:
def model(data):
    features = data.drop(['id', 'product'], axis=1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, random_state=12345)
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = mean_squared_error(target_valid, predictions_valid) ** 0.5
    print('RMSE: ', result)
    print("Средний запас предсказанного сырья: ", predictions_valid.mean())
    target_valid = target_valid.reset_index(drop=True)
    return predictions_valid, target_valid

Так как нужны 3 модели, то написал функцию для упрощения

In [5]:
predictions_data_0, target_data_0 = model(data_0)

RMSE:  37.5794217150813
Средний запас предсказанного сырья:  92.59256778438038


По первой таблице модель предсказала значения с отклонением, в среднем, на 37 тысяч баррелей.

In [6]:
predictions_data_1, target_data_1 = model(data_1)

RMSE:  0.893099286775616
Средний запас предсказанного сырья:  68.728546895446


По второй таблице модель предсказала значения с очень низким отклонением, в среднем, на 0.89 тысяч баррелей.

In [7]:
predictions_data_2, target_data_2 = model(data_2)

RMSE:  40.02970873393434
Средний запас предсказанного сырья:  94.96504596800489


По третьей таблице модель предсказала значения с отклонением, в среднем, на 40 тысяч баррелей.

# Вывод:

Наименьшее отклонение получилось для второй таблицы. У нее же и наименьший средний запас предсказанного сырья.
Возможно, это связано с сильной корреляцией f2 и целевого признака.

## Подготовка к расчёту прибыли

3.1. Все ключевые значения для расчётов сохраните в отдельных переменных.

In [8]:
MONEY_FOR_DEVELOPMENT = 10000000000                              # Денег на разработку скважин в регионе, руб.
BEST_OIL_FIELD = 200                                             # Лучших скважин в регионе (по условию), шт.
MONEY_FOR_ONE_OIL_FIELD = MONEY_FOR_DEVELOPMENT / BEST_OIL_FIELD # Денег на разработку одной скважины, руб.
COST_PER_UNIT_OF_PRODUCT = 450000                                # стоимость единицы продукта, руб.

3.2. Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.

In [9]:
print('Достаточный объём сырья для безубыточной разработки новой скважины: ', 
     MONEY_FOR_ONE_OIL_FIELD / COST_PER_UNIT_OF_PRODUCT)

Достаточный объём сырья для безубыточной разработки новой скважины:  111.11111111111111


# Вывод:

Для безубыточной разработки скважины необходимо не менее 111.2 тыс. баррелей нефти в месторождении и видно, что модель напредсказывала средний запас меньше необходимо во всех трёх регионах.

## Расчёт прибыли и рисков 

4.1. Выберите скважины с максимальными значениями предсказаний.

4.2. Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.

4.3. Рассчитайте прибыль для полученного объёма сырья.

5.1. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.

5.2. Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.

5.3. Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

Функция для расчета прибыли

In [10]:
def revenue(target_valid, predictions_valid, count):
    predict_sorted = predictions_valid.sort_values(ascending=False)
    selected = target_valid[predict_sorted.index][:count]
    return COST_PER_UNIT_OF_PRODUCT * selected.sum() - MONEY_FOR_DEVELOPMENT

Функция применения техники бутестрепа с 1000 выборок по 500 месторождений по 200 лучших месторождений, нахождения средней прибыли, 95% доверительного интервала и риска убытков.

In [11]:
def sample(target_valid, predictions_valid):

    state = np.random.RandomState(12345)
    values = []
    for i in range(1000):
        target_subsample = pd.Series(target_valid).sample(n=500, replace=True, random_state=state)
        predict_subsample = pd.Series(predictions_valid)[target_subsample.index]
        values.append(revenue(target_subsample, predict_subsample, 200))

    values = pd.Series(values)

    loss = values[values < 0].count()
    percent_loss = loss / len(values) * 100
    
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)

    print("Средняя прибыль: ", values.mean().astype(int),",",
           "95% доверительный интервал: ", lower.astype(int), "---", upper.astype(int), ",",
           "Риск убытков: ", percent_loss)

In [12]:
sample(target_data_0, predictions_data_0)

Средняя прибыль:  425938526 , 95% доверительный интервал:  -102090094 --- 947976353 , Риск убытков:  6.0


In [13]:
sample(target_data_1, predictions_data_1)

Средняя прибыль:  515222773 , 95% доверительный интервал:  68873225 --- 931547591 , Риск убытков:  1.0


In [14]:
sample(target_data_2, predictions_data_2)

Средняя прибыль:  435008362 , 95% доверительный интервал:  -128880547 --- 969706954 , Риск убытков:  6.4


# Вывод:

Для окупаемости одного месторождения необходимо минимум 111.2 тыс. баррелей нефти.

Модель для всех трёх регионов предсказала средний запас сырья ниже минимально необходимого.

На данном этапе можно сказать, что если пытаться наугад бурить, то можно легко остаться без денег, но это без бутстрепа.

С помощью бутстрепа удалось выяснить, что наилучшим регионом для разработки скважин является второй.

Несмотря на то, что средний запас сырья модель там предсказала минимальный, риск убытков, равный 1% говорит об обратном, как и наибольшая средняя прибыль, 95% доверительный интервал тоже включает только прибыль, в отличии от остальных регионов, где в доверительном интервале есть и убытки.

По условию необходимо было найти регион с риском убытков меньше 2.5%. Такой регион - 2.